# Vicuna 圧縮テスト
Vicuna_13bを無理やり圧縮して使う

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
from torch import bfloat16
import transformers
from string import Template
from datasets import Dataset
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### データセット読み込み


In [2]:
df = pd.read_csv('./Data/newdf.csv')
df.head()

,userid,course_number,question_number,question_content,text,label,label_mapped
0,D-2020_U51,1,1,今日の内容を自分なりの言葉で説明してみてください,黄色のｍoodleにログインしてそのあと水色のmoodleにリンク授業開始までにteamsの...,4,F
1,D-2020_U51,1,2,今日の内容で、分かったこと・できたことを書いてください,授業の進め方が理解できた,4,F
2,D-2020_U51,1,5,今日の授業の感想や反省を書いてください,オンライン授業ならではの利点を活用する工夫で最初は大丈夫かと不安だったが何とかなりそうだと感じた,4,F
3,D-2020_U45,1,1,今日の内容を自分なりの言葉で説明してみてください,講義の進め方のガイダンス,1,B
4,D-2020_U45,1,2,今日の内容で、分かったこと・できたことを書いてください,講義概要\r\nbookrollの操作方法\r\n講義の進め方\r\n講義時間以外でやるべきこと,1,B


In [3]:
questions = []
for i in range(df["question_number"].unique().shape[0]):
    # print(f"index: {i}")
    # print(f"question text: {df['question_content'][i]}")
    question = {"id": i, "question": df["question_content"][i]}
    questions.append(question)

questions

[{'id': 0, 'question': '今日の内容を自分なりの言葉で説明してみてください'},
 {'id': 1, 'question': '今日の内容で、分かったこと・できたことを書いてください'},
 {'id': 2, 'question': '今日の授業の感想や反省を書いてください'},
 {'id': 3, 'question': '今日の内容を自分なりの言葉で説明してみてください'},
 {'id': 4, 'question': '今日の内容で、分かったこと・できたことを書いてください'},
 {'id': 5, 'question': '今日の授業の感想や反省を書いてください'},
 {'id': 6, 'question': '今日の内容を自分なりの言葉で説明してみてください'},
 {'id': 7, 'question': '今日の内容で、分かったこと・できたことを書いてください'}]

In [4]:
# 講義番号，質問番号ごとに列を作成
def create_column_name(row):
    return f"text-{row['question_number']}"

# 新しい列
df["new_column"] = df.apply(create_column_name, axis=1)

# 新しいデータフレームを定義
df_pivoted = df.pivot_table(
    index=["userid", "course_number", "label", "label_mapped"],
    columns="new_column",
    values="text",
    aggfunc=lambda x: " ".join(str(item) for item in x),
    # aggfunc=lambda x: " ".join(x),
).reset_index()
df = df_pivoted.copy()
df = df.drop(["text-6", "text-7", "text-8"], axis=1)

In [5]:
df_pivoted.head()

new_column,userid,course_number,label,label_mapped,text-1,text-2,text-3,text-4,text-5,text-6,text-7,text-8
0,B-2020_U1,1,4,F,オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業へ...,ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性...,事前の学習で線を引くなどの予習ができていなかったので次回からは授業前にしっかり資料を確認したい,NaN,オンライン授業ではあるが集中力など切れないように今後も学習していきたいと思う,NaN,NaN,NaN
1,B-2020_U10,1,1,B,授業の説明とサイバーセキュリティの現状\r\n,パスワードの使い回しや情報提供に気を付ける\r\nサプライチェーンを利用した攻撃が増えている...,NaN,NaN,スライドを見ながら話を聞いていても知らない横文字言葉が出てくると一時的に思考停止してしまった,NaN,NaN,NaN
2,B-2020_U10,2,1,B,自分の端末の管理は非常に重要で暗号化やバックアップなどが大切\r\nなりすましや偽リンクなど...,秘密の質問で質問に関係ない答えを入力するということを初めて知ってめちゃめちゃびっくりした\r...,NaN,NaN,前日に予習して資料の多さに頭がパンクしそうだったがおかげで大部分が理解できた,NaN,NaN,NaN
3,B-2020_U10,3,1,B,研究における不正行為がどんなものであるかと九州大学が定めている情報倫理について,データの捏造や改竄盗用は絶対に行ってはいけない不正行為であるということ\r\n引用する際は参...,途中すこしだけ寝落ちしてしまった,NaN,前回前々回よりも難易度が上がり予習の段階でよくわからない部分が多々あった,NaN,NaN,NaN
4,B-2020_U10,4,1,B,暗号化複合化の仕組みとセキュリティ証明の仕組みについて,暗号化にもいろいろな方法があることを知った\r\nセキュリティの安全性を証明するために様々な...,NaN,NaN,近所で小学生たちがギャーギャー騒ぎながら遊んでいたので少し集中できなかった,NaN,NaN,NaN


In [6]:
# dataset = load_dataset("kunishou/J-ResearchCorpus", split='train')
# print(dataset[0])
dataset = Dataset.from_pandas(df)

In [7]:
dataset[0]

{'userid': 'B-2020_U1',
 'course_number': 1,
 'label': 4,
 'label_mapped': 'F',
 'text-1': 'オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業への意欲が高まった',
 'text-2': 'ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性を学んだ',
 'text-3': '事前の学習で線を引くなどの予習ができていなかったので次回からは授業前にしっかり資料を確認したい',
 'text-4': None,
 'text-5': 'オンライン授業ではあるが集中力など切れないように今後も学習していきたいと思う'}

### モデル読み込み

In [8]:
model_name = "helloollel/vicuna-7b"
model_id = "lmsys/vicuna-13b-v1.5"
model_id = "elyza/Llama-3-ELYZA-JP-8B"
# model_id = "eachadea/legacy-vicuna-13b"

In [9]:
#! set quantization configuration to load large model with less GPU memory
#! this requires the `bitsandbytes` library

device = "cuda:0" if torch.cuda.is_available() else "cpu"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [10]:
# Load model directly


# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)

model.eval()

tokenizer_config.json: 100%|██████████| 51.0k/51.0k [00:00<00:00, 346kB/s]
tokenizer.json: 100%|██████████| 9.09M/9.09M [00:01<00:00, 7.39MB/s]
special_tokens_map.json: 100%|██████████| 296/296 [00:00<00:00, 1.35MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100%|██████████| 718/718 [00:00<00:00, 3.56MB/s]
model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 29.7MB/s]
model-00001-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [00:56<00:00, 88.2MB/s]
model-00002-of-00004.safetensors: 100%|██████████| 5.00G/5.00G [00:58<00:00, 85.9MB/s]
model-00003-of-00004.safetensors: 100%|██████████| 4.92G/4.92G [00:57<00:00, 85.5MB/s]
model-00004-of-00004.safetensors: 100%|██████████| 1.17G/1.17G [00:13<00:00, 85.7MB/s]
generation_config.json: 100%|██████████| 194/194 [00:00<00:00, 491kB/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

### モデルのテスト
今回は要約タスクをさせます．

テスト用：

In [ ]:
preamble = '''### 以下に論文を示します。
論文の内容を要約してください。
'''

output_prompt = '### 出力：'

template = Template('''$preamble
             
$text

$output_prompt
''')

In [ ]:
def format_input(dataset, idx):
    
    # question = dataset[idx]['question_content']
    text = dataset[idx]['text']

    input_text = template.substitute(
        preamble=preamble,
        text = text,
        output_prompt=output_prompt
    )
    return input_text

In [11]:
preamble = '''### 以下に示すものは、講義「情報科学」に関するアンケートと、回答した生徒の成績です。
アンケートは全部で5問あります。
最初の行が質問文、次の文が生徒の回答文、最後の行が成績です。
成績はA, B, C, D, Fのいずれかです。
アンケートの回答から、生徒の成績の理由を解説してください。'''

output_prompt = '### 出力：'

template = Template('''$preamble

質問1: $question1
回答1: $answer1

質問2: $question2
回答2: $answer2

質問3: $question3
回答3: $answer3

質問4: $question4
回答4: $answer4

質問5: $question5
回答5: $answer5
                    
成績：$grade

$output_prompt
''')

In [12]:
def format_input(dataset, idx):
    
    # question = dataset[idx]['question_content']
    # answer = dataset[idx]['text']

    q1 = questions[0]['question']
    q2 = questions[1]['question']
    q3 = questions[2]['question']
    q4 = questions[3]['question']
    q5 = questions[4]['question']

    p1 = dataset[idx]['text-1']
    p2 = dataset[idx]['text-2']
    p3 = dataset[idx]['text-3']
    p4 = dataset[idx]['text-4']
    p5 = dataset[idx]['text-5']

    grade = dataset[idx]['label_mapped']


    # input_text = template.substitute(
    #     preamble=preamble, question=Q, answer=P, output_prompt=output_prompt)
    input_text = template.substitute(
        preamble=preamble,
        question1=q1, answer1=p1,
        question2=q2, answer2=p2,
        question3=q3, answer3=p3,
        question4=q4, answer4=p4,
        question5=q5, answer5=p5,
        grade=grade,
        output_prompt=output_prompt
    )
    return input_text

In [13]:
# example
print(format_input(dataset, 0))

### 以下に示すものは、講義「情報科学」に関するアンケートと、回答した生徒の成績です。
アンケートは全部で5問あります。
最初の行が質問文、次の文が生徒の回答文、最後の行が成績です。
成績はA, B, C, D, Fのいずれかです。
アンケートの回答から、生徒の成績の理由を解説してください。

質問1: 今日の内容を自分なりの言葉で説明してみてください
回答1: オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業への意欲が高まった

質問2: 今日の内容で、分かったこと・できたことを書いてください
回答2: ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性を学んだ

質問3: 今日の授業の感想や反省を書いてください
回答3: 事前の学習で線を引くなどの予習ができていなかったので次回からは授業前にしっかり資料を確認したい

質問4: 今日の内容を自分なりの言葉で説明してみてください
回答4: None

質問5: 今日の内容で、分かったこと・できたことを書いてください
回答5: オンライン授業ではあるが集中力など切れないように今後も学習していきたいと思う
                    
成績：F

### 出力：



In [14]:
def summarize_text(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=2048, truncation=True)
    print(f"input text:{tokenizer.decode(inputs[0])}")
    print(f"input tensor length:{len(inputs[0])}")
    inputs.to('cuda:0')


    generation_config = GenerationConfig(
    max_new_tokens=1000,
    min_length=40,
    length_penalty=0.5,
    num_beams=4,
    early_stopping=True
    )
    outputs = model.generate(inputs, generation_config=generation_config)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [15]:
print(dataset[0]['label_mapped'])
# sample_text = dataset[0]['text']
summary = summarize_text(format_input(dataset, 0))


F
input text:<|begin_of_text|>### 以下に示すものは、講義「情報科学」に関するアンケートと、回答した生徒の成績です。
アンケートは全部で5問あります。
最初の行が質問文、次の文が生徒の回答文、最後の行が成績です。
成績はA, B, C, D, Fのいずれかです。
アンケートの回答から、生徒の成績の理由を解説してください。

質問1: 今日の内容を自分なりの言葉で説明してみてください
回答1: オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業への意欲が高まった

質問2: 今日の内容で、分かったこと・できたことを書いてください
回答2: ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性を学んだ

質問3: 今日の授業の感想や反省を書いてください
回答3: 事前の学習で線を引くなどの予習ができていなかったので次回からは授業前にしっかり資料を確認したい

質問4: 今日の内容を自分なりの言葉で説明してみてください
回答4: None

質問5: 今日の内容で、分かったこと・できたことを書いてください
回答5: オンライン授業ではあるが集中力など切れないように今後も学習していきたいと思う
                    
成績：F

### 出力：

input tensor length:358


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(summary)

### 以下に示すものは、講義「情報科学」に関するアンケートと、回答した生徒の成績です。
アンケートは全部で5問あります。
最初の行が質問文、次の文が生徒の回答文、最後の行が成績です。
成績はA, B, C, D, Fのいずれかです。
アンケートの回答から、生徒の成績の理由を解説してください。

質問1: 今日の内容を自分なりの言葉で説明してみてください
回答1: オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業への意欲が高まった

質問2: 今日の内容で、分かったこと・できたことを書いてください
回答2: ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性を学んだ

質問3: 今日の授業の感想や反省を書いてください
回答3: 事前の学習で線を引くなどの予習ができていなかったので次回からは授業前にしっかり資料を確認したい

質問4: 今日の内容を自分なりの言葉で説明してみてください
回答4: None

質問5: 今日の内容で、分かったこと・できたことを書いてください
回答5: オンライン授業ではあるが集中力など切れないように今後も学習していきたいと思う
                    
成績：F

### 出力：
質問1: 今日の内容を自分なりの言葉で説明してみてください
回答1: オンライン授業での初回ということもあり不安な点はあったが今後の進め方についても理解でき授業への意欲が高まった

質問2: 今日の内容で、分かったこと・できたことを書いてください
回答2: ipaという組織がセキュリティに関する情報を発信していることやパスワードを変えることの重要性を学んだ

質問3: 今日の授業の感想や反省を書いてください
回答3: 事前の学習で線を引くなどの予習ができていなかったので次回からは授
